表1
|DAN|DAN预激活的KC|
|:--:| :--: |
|0	|0，1|
|1	|2，3|
|2	|4，5|

### 已经实现
1. 随机让KC，MBON激活，这一步是训练KC到MBON的权重
   + 同时兴奋，有奖励，KM突触连接权重增强
   + 不同时兴奋，有奖励，KM突触连接权重减弱
   + 无奖励，KM突触连接权重不变
   + τdw/dt=(α * V_前  − β * w)* V_后* R, α>=β
2. KC输出兴奋程度的输入来源有两方面   1.sensor输入 2.DAN预激活
3. MBON激活程度 = KC * WeightKM
4. MBON达到激活阈值，对应DAN也激活
5. DAN激活，会导致KC预激活（额外）,如表1

### 问题
+ DAN到KC的权重如何更新
+ KC到DAN的权重如何更新
  + 如何根据KC增量，激活DAN

In [ ]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-(np.array(x)-2)))


def rk4(h, y, inputs, f):
    '''
    用于数值积分的rk4函数。
    args:
        h - 步长
        y - 当前状态量
        inputs - 外界对系统的输入
        f - 常微分或偏微分方程
    return:
        y_new - 新的状态量,即经过h时间之后的状态量
    '''
    k1 = f(y, inputs)
    k2 = f(y + h / 2 * k1, inputs)
    k3 = f(y + h / 2 * k2, inputs)
    k4 = f(y + h * k3, inputs)

    y_new = y + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return y_new


class KMDmodel:
    def __init__(self, numKC=6,numMBON=3,numDAN=3) -> None:
        # sensor输入
        self.inputsensor = np.zeros(numKC,dtype=float)
        # DAN预激活KC
        self.DANpotentialKC = {'0':[0,1],'1':[2,3],'2':[4,5]}
        # MBON的激活阈值
        self.thresholdMBON = 0.5
        # 当前兴奋程度
        self.KC = np.zeros(numKC,dtype=float)
        self.MBON = np.zeros(numMBON,dtype=float)
        self.DAN = np.zeros(numDAN,dtype=float)
    
    def outputKC(self, inputsensor, inputDAN):
        """
        KC输出兴奋程度来源有两方面
        1.sensor输入 2.DAN预激活
        """
        pokc = []
        for i, value in enumerate(inputDAN):
            if value == 1:
                pokc.append(self.DANpotentialKC[str(i)])
        for i in pokc:
            for j in i:
                self.KC[j] += 1
        self.KC = sigmoid(inputsensor + self.KC)        
        return self.KC
    
    def outputMBON(self, weightKM):
        """
        MBON输出兴奋程度 = KC * Weight_KM
        """
        self.MBON = sigmoid(np.dot(self.KC,weightKM))
        return self.MBON

    def outputDAN(self):
        """
        DAN输出兴奋程度，先简化为对应的MBON到达激活阈值，DAN就激活
        """
        for index, value in enumerate(self.MBON):
            if value >= self.thresholdMBON:
                self.DAN[index] = 1
        return self.DAN
    

class Synapse:
    def __init__(self, numPreNeuron, numPostNeuron) -> None:
        self.numPreNeuron = numPreNeuron
        self.numPostNeuron = numPostNeuron
        self.PreNeuron = np.array([np.zeros(numPreNeuron, dtype=float)])
        self.PostNeruon = np.array([np.zeros(numPostNeuron, dtype=float)])
        self.weight = np.zeros([numPreNeuron,numPostNeuron])
        self.reward = np.zeros(1)
        # 超参数
        self.alpha = 1
        self.tau = 1
        self.beta = 1
        self.dt = 1
    
    def derivative(self, state, inputs=0):
        w = state
        Dw = (np.matmul(self.alpha * (np.tile(self.PreNeuron.T, self.numPostNeuron) - w), self.beta * np.tile(self.PostNeruon.T, self.numPostNeuron)) * self.reward) / self.tau
        return Dw
    
    def step(self, state, dt=1, inputs=0):
        statenew = rk4(dt, state, inputs, self.derivative)
        return statenew
    
    
# if __name__  == "__main__":
kmd = KMDmodel()
kmd.MBON = [0,1,0.5]
inputsensor = [0,1,1,1,1,1]
kmd.outputKC(inputsensor,kmd.outputDAN())

KCtoMBON = Synapse(6,3)
w = KCtoMBON.step(KCtoMBON.weight)
kmd.outputMBON(w)
